## Load_Env & Set_Size

In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"

from procedural_envs import composer
from procedural_envs.misc.observers import GraphObserver, EXTRA_ROOT_NODE_DICT
from procedural_envs.tasks.observation_config import obs_config_dict

env_0 = 'ant_touch_4'
obs_config_0 = 'amorpheus'
obs_config = obs_config_dict[obs_config_0]

if ('handsup2' in env_0) and ('ant' in env_0):
    obs_config['extra_root_node'] = EXTRA_ROOT_NODE_DICT['ant_handsup2']
elif ('handsup2' in env_0) and ('centipede' in env_0):
    obs_config['extra_root_node'] = EXTRA_ROOT_NODE_DICT['centipede_handsup2']
elif ('handsup' in env_0) and ('unimal' in env_0):
    obs_config['extra_root_node'] = EXTRA_ROOT_NODE_DICT['unimal_handsup']
elif 'handsup' in env_0:
    obs_config['extra_root_node'] = EXTRA_ROOT_NODE_DICT['handsup']

observer = GraphObserver(name=obs_config_0, **obs_config)
env_fn = composer.create_fn(env_0, observer=observer, observer2=observer)
env = env_fn()
local_state_size = env.observation_size // env.num_node
observation_size = env.observation_size
num_limb = observation_size // local_state_size
action_size = env.action_size
state_qp_shape = num_limb + 1
# if is_claw:
#   state_qp_shape = num_limb + 1 - int(action_size / 4)  # int(action_size / 4) == num_legs
print("local_state_size:", local_state_size)
print("observation_size:", observation_size)
print("num_limb:", num_limb)
print("action_size:", action_size)
print("state_qp_shape:",state_qp_shape)
print(type(env))

[-0.7361131  3.8408964  0.25     ]


[-0.7361131  3.8408964  0.25     ]


[-0.7361131  3.8408964  0.25     ]
local_state_size: 20
observation_size: 200
num_limb: 10
action_size: 8
state_qp_shape: 11
<class 'procedural_envs.misc.env_wrapper.GoalEvalWrapper'>


## Get_Significant_Info

In [2]:
from brax import jumpy as jp

cus_state = env.reset(rng=jp.random_prngkey(seed=0))
cus_obs = env._get_obs(cus_state.qp, cus_state.info)

len(cus_obs[0]['amorpheus'])

[-0.7361131  3.8408964  0.25     ]


200

In [3]:
child_dict = {j.child: (i, j.name, j.parent) for i, j in enumerate(env.sys.config.joints)}
actuator_dict = {a.joint: (i, a.name, a.strength) for i, a in enumerate(env.sys.config.actuators)}

print("child_dict: ", child_dict)

child_dict:  {'agent1___Aux 1_0': (0, 'agent1___torso_Aux 1_0', 'agent1___torso'), 'agent1___$ Body 4_0': (1, 'agent1___Aux 1_$ Body 4_0', 'agent1___Aux 1_0'), 'agent1___Aux 1_1': (2, 'agent1___torso_Aux 1_1', 'agent1___torso'), 'agent1___$ Body 4_1': (3, 'agent1___Aux 1_$ Body 4_1', 'agent1___Aux 1_1'), 'agent1___Aux 1_2': (4, 'agent1___torso_Aux 1_2', 'agent1___torso'), 'agent1___$ Body 4_2': (5, 'agent1___Aux 1_$ Body 4_2', 'agent1___Aux 1_2'), 'agent1___Aux 1_3': (6, 'agent1___torso_Aux 1_3', 'agent1___torso'), 'agent1___$ Body 4_3': (7, 'agent1___Aux 1_$ Body 4_3', 'agent1___Aux 1_3')}


In [4]:
%load_ext autoreload
%autoreload 2

from procedural_envs.misc import observers

obs_dict = observers.get_obs_dict(
        env.sys, cus_state.qp, cus_state.info, env.observer, {}, env.metadata.components['agent1'])
env.observer._get_obs(env.sys, cus_state.qp, cus_state.info, {}, env.metadata.components['agent1'])
print("env.observer.type: ", type(env.observer))
print(env.sys.body.index)

env.observer.type:  <class 'procedural_envs.misc.observers.GraphObserver'>
{'agent1___torso': 0, 'agent1___Aux 1_0': 1, 'agent1___$ Body 4_0': 2, 'agent1___Aux 1_1': 3, 'agent1___$ Body 4_1': 4, 'agent1___Aux 1_2': 5, 'agent1___$ Body 4_2': 6, 'agent1___Aux 1_3': 7, 'agent1___$ Body 4_3': 8, 'cap1___Ball': 9, 'Ground': 10}


In [5]:
env.observation_size

[-0.7361131  3.8408964  0.25     ]


200

In [6]:
from procedural_envs.misc import component_editor
from procedural_envs.components import *
from procedural_envs import composer
import importlib
import json



new_v = {}
new_v['json'] = component_editor.message_str2json(ant.get_specs(4)['message_str'])

new_v['json']
# print("num_bodies: ", len(new_v['json']['bodies']))
# print("num_joints: ", len(new_v['json']['joints']))

{'bodies': [{'name': 'torso',
   'colliders': [{'capsule': {'radius': 0.25, 'length': 0.5, 'end': 1}}],
   'inertia': {'x': 1.0, 'y': 1.0, 'z': 1.0},
   'mass': 10.0},
  {'name': 'Aux 1_0',
   'colliders': [{'rotation': {'x': 90.0, 'y': -90.0},
     'capsule': {'radius': 0.08, 'length': 0.44284272}}],
   'inertia': {'x': 1.0, 'y': 1.0, 'z': 1.0},
   'mass': 1.0},
  {'name': '$ Body 4_0',
   'colliders': [{'rotation': {'x': 90.0, 'y': -90.0},
     'capsule': {'radius': 0.08, 'length': 0.7256854, 'end': -1}}],
   'inertia': {'x': 1.0, 'y': 1.0, 'z': 1.0},
   'mass': 1.0},
  {'name': 'Aux 1_1',
   'colliders': [{'rotation': {'x': 90.0, 'y': -90.0},
     'capsule': {'radius': 0.08, 'length': 0.44284272}}],
   'inertia': {'x': 1.0, 'y': 1.0, 'z': 1.0},
   'mass': 1.0},
  {'name': '$ Body 4_1',
   'colliders': [{'rotation': {'x': 90.0, 'y': -90.0},
     'capsule': {'radius': 0.08, 'length': 0.7256854, 'end': -1}}],
   'inertia': {'x': 1.0, 'y': 1.0, 'z': 1.0},
   'mass': 1.0},
  {'name': 'Au

## Load_qp && Save_Nodes

In [7]:
import pickle
saved_qp = "../data/ant_touch_4_qp.pkl"
with open(saved_qp, 'rb') as f:
    saved_qp = pickle.load(f)
saved_qp.shape

(1, 97900, 143)

In [8]:
import brax
from brax.io import image
from IPython.display import Image
def get_obs2(s_qp, env):
    s_info = env.sys.info(s_qp)
    obs_dict, _ = env._get_obs2(s_qp, s_info)
    obs = data_utils.concat_array(obs_dict, env.observer_shapes2)
    return obs

def qp_convert(s_qp, state_qp_shape):
    return brax.QP(
      pos=s_qp[:, 0:state_qp_shape * 3].reshape(-1, state_qp_shape, 3),
      rot=s_qp[:, state_qp_shape * 3:state_qp_shape * 7].reshape(-1, state_qp_shape, 4),
      vel=s_qp[:, state_qp_shape * 7:state_qp_shape * 10].reshape(-1, state_qp_shape, 3),
      ang=s_qp[:, state_qp_shape * 10:state_qp_shape * 13].reshape(-1, state_qp_shape, 3))

def qp_squeeze(s_qp):
    return brax.QP(
      pos=s_qp.pos[0],
      rot=s_qp.rot[0],
      vel=s_qp.vel[0],
      ang=s_qp.ang[0])

converted_qp = [qp_squeeze(qp_convert(saved_qp[0, t:t+1], state_qp_shape)) for t in range(0, 500)]

In [20]:
saved_qp[0][0]

array([ 0.0000000e+00,  0.0000000e+00,  5.1334018e-01,  3.0142137e-01,
        0.0000000e+00,  5.1334018e-01,  6.2465054e-01,  0.0000000e+00,
        2.9667008e-01,  1.8456735e-17,  3.0142137e-01,  5.1334018e-01,
        1.8456735e-17,  6.2465054e-01,  2.9667008e-01, -3.0142137e-01,
        3.6913471e-17,  5.1334018e-01, -6.2465054e-01,  7.6497630e-17,
        2.9667008e-01, -5.5370206e-17, -3.0142137e-01,  5.1334018e-01,
       -5.5370206e-17, -6.2465054e-01,  2.9667008e-01,  4.0840912e+00,
       -2.5589771e+00,  2.5000000e-01,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  1.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  1.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  9.0630782e-01,  0.0000000e+00,  4.2261824e-01,
        0.0000000e+00,  7.0710677e-01,  0.0000000e+00,  0.0000000e+00,
        7.0710677e-01,  6.4085639e-01, -2.9883623e-01,  2.9883623e-01,
        6.4085639e-01,  6.1232343e-17,  0.0000000e+00,  0.0000000e+00,
      

## Make_Graph

In [10]:
my_obs = env.observer._get_obs(env.sys, converted_qp[23], cus_state.info, {}, env.metadata.components['agent1'])

In [11]:
converted_qp[0]

QP(pos=array([[ 0.0000000e+00,  0.0000000e+00,  5.1334018e-01],
       [ 3.0142137e-01,  0.0000000e+00,  5.1334018e-01],
       [ 6.2465054e-01,  0.0000000e+00,  2.9667008e-01],
       [ 1.8456735e-17,  3.0142137e-01,  5.1334018e-01],
       [ 1.8456735e-17,  6.2465054e-01,  2.9667008e-01],
       [-3.0142137e-01,  3.6913471e-17,  5.1334018e-01],
       [-6.2465054e-01,  7.6497630e-17,  2.9667008e-01],
       [-5.5370206e-17, -3.0142137e-01,  5.1334018e-01],
       [-5.5370206e-17, -6.2465054e-01,  2.9667008e-01],
       [ 4.0840912e+00, -2.5589771e+00,  2.5000000e-01],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32), rot=array([[ 1.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 1.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 9.0630782e-01,  0.0000000e+00,  4.2261824e-01,  0.0000000e+00],
       [ 7.0710677e-01,  0.0000000e+00,  0.0000000e+00,  7.0710677e-01],
       [ 6.4085639e-01, -2.9883623e-01,  2.9883623e-0

In [22]:
for i in range(300):
    cus_state.done

0.0

In [13]:
new_v['json']

{'bodies': [{'name': 'torso',
   'colliders': [{'capsule': {'radius': 0.25, 'length': 0.5, 'end': 1}}],
   'inertia': {'x': 1.0, 'y': 1.0, 'z': 1.0},
   'mass': 10.0},
  {'name': 'Aux 1_0',
   'colliders': [{'rotation': {'x': 90.0, 'y': -90.0},
     'capsule': {'radius': 0.08, 'length': 0.44284272}}],
   'inertia': {'x': 1.0, 'y': 1.0, 'z': 1.0},
   'mass': 1.0},
  {'name': '$ Body 4_0',
   'colliders': [{'rotation': {'x': 90.0, 'y': -90.0},
     'capsule': {'radius': 0.08, 'length': 0.7256854, 'end': -1}}],
   'inertia': {'x': 1.0, 'y': 1.0, 'z': 1.0},
   'mass': 1.0},
  {'name': 'Aux 1_1',
   'colliders': [{'rotation': {'x': 90.0, 'y': -90.0},
     'capsule': {'radius': 0.08, 'length': 0.44284272}}],
   'inertia': {'x': 1.0, 'y': 1.0, 'z': 1.0},
   'mass': 1.0},
  {'name': '$ Body 4_1',
   'colliders': [{'rotation': {'x': 90.0, 'y': -90.0},
     'capsule': {'radius': 0.08, 'length': 0.7256854, 'end': -1}}],
   'inertia': {'x': 1.0, 'y': 1.0, 'z': 1.0},
   'mass': 1.0},
  {'name': 'Au

In [14]:
import numpy as np
from xvfbwrapper import Xvfb
vdisplay = Xvfb()
vdisplay.start()


In [15]:
actuable_nodes = []
for i in range(len(new_v['json']['actuators'])):
    actuable_nodes.append(new_v['json']['actuators'][i]['name'])
actuable_nodes

['torso_Aux 1_0',
 'Aux 1_$ Body 4_0',
 'torso_Aux 1_1',
 'Aux 1_$ Body 4_1',
 'torso_Aux 1_2',
 'Aux 1_$ Body 4_2',
 'torso_Aux 1_3',
 'Aux 1_$ Body 4_3']

In [16]:
edges = []
for i in range(len(new_v['json']['joints'])):
    edges.append([new_v['json']['joints'][i]['parent'], new_v['json']['joints'][i]['child']])
edges

[['torso', 'Aux 1_0'],
 ['Aux 1_0', '$ Body 4_0'],
 ['torso', 'Aux 1_1'],
 ['Aux 1_1', '$ Body 4_1'],
 ['torso', 'Aux 1_2'],
 ['Aux 1_2', '$ Body 4_2'],
 ['torso', 'Aux 1_3'],
 ['Aux 1_3', '$ Body 4_3']]

In [17]:
nodes = []
for i in range(len(new_v['json']['bodies'])):
    nodes.append(new_v['json']['bodies'][i]['name'])
nodes

['torso',
 'Aux 1_0',
 '$ Body 4_0',
 'Aux 1_1',
 '$ Body 4_1',
 'Aux 1_2',
 '$ Body 4_2',
 'Aux 1_3',
 '$ Body 4_3']

In [18]:
node_dict = {}
mask_dict = {}
node_ranges = {}
import jax

for idx, _node in enumerate(nodes):
    start_pt = idx * 20
    node_dict[_node] = [jax.device_get(my_obs[start_pt + 16]).item(), jax.device_get(my_obs[start_pt + 19]).item(), 0.0]
    mask_dict[_node] = [True, True, False]
    node_ranges[_node] = [jax.device_get(my_obs[start_pt + 17]).item(), jax.device_get(my_obs[start_pt + 18]).item()]
print("node_dict: ", node_dict)
print("mask_dict: ", mask_dict)
print("node_ranges: ", node_ranges)

node_dict:  {'torso': [0.0, 0.0, 0.0], 'Aux 1_0': [1.0848101377487183, -0.40592458844184875, 0.0], '$ Body 4_0': [1.0128082036972046, -0.10523267835378647, 0.0], 'Aux 1_1': [0.2534720003604889, 4.9936676025390625, 0.0], '$ Body 4_1': [0.058837033808231354, -0.7612283825874329, 0.0], 'Aux 1_2': [-0.09575316309928894, 0.007336159702390432, 0.0], '$ Body 4_2': [0.20657682418823242, -1.1135088205337524, 0.0], 'Aux 1_3': [0.28368911147117615, 7.203074932098389, 0.0], '$ Body 4_3': [0.14208221435546875, -0.6011561155319214, 0.0]}
mask_dict:  {'torso': [True, True, False], 'Aux 1_0': [True, True, False], '$ Body 4_0': [True, True, False], 'Aux 1_1': [True, True, False], '$ Body 4_1': [True, True, False], 'Aux 1_2': [True, True, False], '$ Body 4_2': [True, True, False], 'Aux 1_3': [True, True, False], '$ Body 4_3': [True, True, False]}
node_ranges:  {'torso': [0.0, 0.0], 'Aux 1_0': [0.4166666865348816, 0.5833333730697632], '$ Body 4_0': [0.5833333730697632, 0.694444477558136], 'Aux 1_1': [0.4

In [19]:
actuator_to_nodes = {}
for idx, _actuator in enumerate(actuators):
    actuator_to_nodes[_actuator] = [new_v['json']['joints'][idx]['parent'], new_v['json']['joints'][idx]['child']]
actuator_to_nodes

NameError: name 'actuators' is not defined

In [ ]:
converted_qp[23]

In [ ]:
node_to_actuators = {}
for key in actuator_to_nodes:
    for _node in actuator_to_nodes[key]:
        if _node in node_to_actuators:
            node_to_actuators[_node].append(key)
        else:
            node_to_actuators[_node] = [key]
node_to_actuators

In [ ]:
actuable_nodes = []
for i in range(len(new_v['json']['actuators'])):
    actuable_nodes.append(new_v['json']['actuators'][i]['name'])
actuable_nodes

In [ ]:
env.sys.joints[0].body_p.idx

In [ ]:
env.sys.joints[0].angle_vel(converted_qp[13])[1]

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def vis_graph(nodes, edges, actuable_nodes, labels=None):
    # create directed graph object
    DG = nx.DiGraph()
    DG.add_nodes_from(nodes)
    DG.add_edges_from(edges)
    
    # if self.morphology == 'dog':
    #     figsize = (60, 40)
    #     rotate = 90
    # else:
    #     figsize = (18, 12)
    #     rotate = 0
    figsize = (18, 12)
    rotate = 0
    # set node colors
    color_map = []
    for node in nodes:
        if node in actuable_nodes:
            color_map.append('green')
        else:
            color_map.append('red')

    # layout graph
    plt.figure(figsize=figsize)
    pos = nx.nx_agraph.graphviz_layout(DG, prog="dot")

    # Apply rotation to layout positions
    if rotate != 0:
        rotate = rotate / 180 * np.pi
        pos = {node: (x * np.cos(rotate) - y * np.sin(rotate), x * np.sin(rotate) + y * np.cos(rotate)) for node, (x, y) in pos.items()}

    # draw graph
    nx.draw_networkx(DG, pos, node_color=color_map, labels=labels,
                     node_size=6000, arrowsize=20, font_weight='bold', font_size=10)
    plt.show()

vis_graph(nodes, edges, actuable_nodes)

In [ ]:
def pretty_print_dict(x, prefix=''):
    for k, v in x.items():
        print(f'{prefix}{k:20s}: {v}')


def test_env(domain, task):
    print()
    parser = XmlGraphParser(domain, task)
    parser.env.reset()
    parser.env.step(1)
    obs = parser.env.task.get_observation(parser.env.physics)
    node_dict, mask_dict = parser.parse_observation(obs)
    pretty_print_dict(node_dict)
    labels = {node: node + '\n' + 
                    (parser.node_to_actuators[node] if node in parser.actuable_nodes else '-') + '\n' +
                    (', '.join(['T' if m else 'F' for m in mask_dict[node]]) if node in mask_dict else "") + '\n' +
                    str(nrange)
              for node, nrange in zip(parser.nodes, parser.node_ranges)}
    parser.vis_graph(labels)
